In [37]:
import numpy as np
import pandas as pd
%matplotlib inline 
import re
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
plt.rcParams['figure.figsize'] = (8, 6)
import xlsxwriter
import scipy.stats as stats

In [38]:
#name of experiment
name = 'Multirun_SuggestUsersForChat_run1'
# files with payments of all users
tg = pd.read_csv(r'C:\Users\user\Downloads\report-1524485834.csv')
cg1 = pd.read_csv(r'C:\Users\user\Downloads\report-1524485847.csv')
cg2 = pd.read_csv(r'C:\Users\user\Downloads\report-1524485856.csv')
tg['group'] = 'tg'
cg1['group'] = 'cg1'
cg2['group'] = 'cg2'
df= tg.append(cg1.append(cg2))

In [39]:
df.head()

,user_id,sex,isReferral,isPaid,active_days_0_1day,active_days_1day_2day,active_days_2day_3day,like_sent_0_1day,like_sent_1day_2day,like_sent_2day_3day,...,fist_dialog_received_fake_0_1day,fist_dialog_received_fake_1day_2day,fist_dialog_received_fake_2day_3day,first_dialog_started_fake_0_1day,first_dialog_started_fake_1day_2day,first_dialog_started_fake_2day_3day,payment_0_1day,payment_1day_2day,payment_2day_3day,group
0,117978263,1,0,0,1,0,1,6,0,0,...,0,0,0,0,0,0,0,0,0,tg
1,118028416,1,0,1,1,0,0,7,0,0,...,0,0,0,0,0,0,1,0,0,tg
2,118003862,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,tg
3,118009783,1,0,0,0,0,0,12,0,0,...,0,0,0,0,0,0,0,0,0,tg
4,117991988,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,tg


In [40]:
writer = pd.ExcelWriter(r'C:\Users\user\Downloads\analysis_of_experiment_'+name+'.xlsx', engine='openpyxl') 

In [41]:
df['like_sent-fake_0_1day'] = df['like_sent_0_1day'] - df['fake_like_sent_0_1day']
df['like_sent-fake_1day_2day'] = df['like_sent_1day_2day'] - df['fake_like_sent_1day_2day']
df['like_sent-fake_2day_3day'] = df['like_sent_2day_3day'] - df['fake_like_sent_2day_3day']

df['mutual-fake_0_1day'] = df['mutual_0_1day'] - df['fake_mutual_0_1day']
df['mutual-fake_1day_2day'] = df['mutual_1day_2day'] - df['fake_mutual_1day_2day']
df['mutual-fake_2day_3day'] = df['mutual_2day_3day'] - df['fake_mutual_2day_3day']


In [42]:
def mean_and_part(value, period):
        a = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value + period].mean()).transpose()
        b =(pd.DataFrame(df[(df['active_days' + period] >0) & (df[value + period] >0)].groupby('group')['user_id'].count()).transpose()/
            pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
        return a.append(b)

In [43]:
def mean_and_part2(value, period):
        a = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value + period].mean()).transpose()
        b =(pd.DataFrame(df[(df['active_days'+ period] >0) & (df[value + period] >0)].groupby('group')['user_id'].count()).transpose()/
            pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
        c = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value+ '-fake' + period].mean()).transpose()
        d = pd.DataFrame(df[(df['active_days'+ period] >0) & (df[value+ '-fake' + period] >0)].groupby('group')['user_id'].count().transpose()/
            pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
        return a.append(b.append(c.append(d)))

In [62]:
def mean_and_part3(value, period):
        a = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value + period].mean()).transpose()
        b = pd.DataFrame(df[df[value + period] >0].groupby('group')['user_id'].count()).transpose()/pd.DataFrame(df.groupby('group')['user_id'].count()).transpose()
        return a.append(b)

In [63]:
def summary_statistics(period):
    b = mean_and_part2('mutual', period)
    c = mean_and_part2('like_sent', period)
    d = mean_and_part3('like_received', period)
    e = mean_and_part3('message_sent', period)
    f = mean_and_part3('message_received', period)
    g = mean_and_part3('first_dialog_started', period)
    h = mean_and_part3('fist_dialog_received', period)
    k = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose()
    
    m = pd.concat([b,c,d,e,f,g,h,k],axis = 0, join = 'inner')
    m['dif'] = m['tg']/(m['cg1']+m['cg2'])*2-1
    cols = ['tg', 'cg1', 'cg2', 'dif']
    m = m[cols]
    m.columns = ['TG', 'CG1', 'CG2', 'dif']
    m.index = ['mutual', 'mutual_nonzero_part', 'mutual-fake', 'mutual-fake_nonzero_part',
                'like_sent', 'like_sent_nonzero_part', 'like_sent-fake', 'like_sent-fake_nonzero_part','like_received',
                'like_received_nonzero_part', 'message_sent', 'message_sent_nonzero_part',
                'message_received', 'message_receive_nonzero_part',
                'first_dialog_started', 'first_dialog_started_nonzero_part',
                'fist_dialog_received', 'fist_dialog_received_nonzero_part', 'active_users']
    cm = sns.light_palette("yellow", as_cmap=True)
    return (m.fillna(0))

In [64]:
m1 = summary_statistics('_0_1day')
m2 = summary_statistics('_1day_2day')
m3 = summary_statistics('_2day_3day')

m = pd.concat([m1,m2,m3], axis = 1)
m.columns = ['TG_1', 'CG1_1', 'CG2_1', 'dif_1',
             'TG_2', 'CG1_2', 'CG2_2', 'dif_2',
             'TG_3', 'CG1_3', 'CG2_3', 'dif_3']
cm = sns.light_palette("yellow", as_cmap=True)
m.fillna(0).style.format({'dif_1': '{:.2%}', 'dif_2': '{:.2%}',
                'dif_3': '{:.2%}'}).background_gradient(subset=['dif_1', 'dif_2', 'dif_3'], cmap=cm).to_excel(writer, "Main")

In [66]:
writer.save()